In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "remove_initialisation_effects/"
create_directory(data_directory)
create_directory("models")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 100
NORMALISE_DATA = True
STEP_SIZE = 1
SEQ_END = 2700

INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5")
val_trials = read_dataset("data/val_passive_trials.h5")

100%|██████████| 1000/1000 [00:10<00:00, 97.47it/s]


## Train model and plot loss and accuracy

In [6]:
N_MODELS = 3

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:01<00:00, 668.46it/s]
Train_loss: ([0.8356112  0.89984073]) Train_acc: ([57.37142857 48.94285714]) Val_acc: ([59.  48.7]): 100%|██████████| 100/100 [19:31<00:00, 11.67s/it]
Train_loss: ([0.59038335 0.65417118]) Train_acc: ([67.14285714 59.71428571]) Val_acc: ([56.5 55.1]): 100%|██████████| 100/100 [19:28<00:00, 11.77s/it]
Train_loss: ([0.50793313 0.55336084]) Train_acc: ([76.02857143 69.68571429]) Val_acc: ([58.8 58.6]): 100%|██████████| 100/100 [19:24<00:00, 11.58s/it]
Train_loss: ([0.59642567 0.684926  ]) Train_acc: ([65.94285714 56.11428571]) Val_acc: ([55.9 49.4]): 100%|██████████| 100/100 [19:19<00:00, 11.78s/it]
Train_loss: ([0.74724249 0.83186054]) Train_acc: ([56.11428571 48.6       ]) Val_acc: ([50.9 43.9]): 100%|██████████| 100/100 [19:28<00:00, 11.91s/it]


## Save model and scaler

In [7]:
joblib.dump(scaler, "scalers/passive_dual_scaler.sk")

['scalers/passive_dual_scaler.sk']